In [1]:
import sympy as sp
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)
from sympy.physics.mechanics import dynamicsymbols, Point, ReferenceFrame
import xml.etree.ElementTree as ET
tree = ET.parse('model.xml')
root = tree.getroot()
from lxml.etree import fromstring

links = []
joints = []
for item in root.findall('./model/'):
    if item.tag == 'joint':
        joints.append(item.attrib['name'])
for item in root.findall('./model/'):
    if item.tag == 'link':
        links.append(item.attrib['name'])
keys = ['x','y','z','roll','pitch','yaw']
values = []
for x in joints:
    for item in root.findall('./model/'):
        if 'name' in item.attrib and item.attrib['name'] == x:
            for child in item:
                if child.tag == 'pose':
                    values.append(child.text.split())
linkvalues = []
for x in links:
    for item in root.findall('./model/'):
        if 'name' in item.attrib and item.attrib['name'] == x:
            for child in item:
                if child.tag == 'pose':
                    linkvalues.append(child.text.split())
linkposes = dict(zip(links, map(list, (zip(*linkvalues)))))
lin = dict(zip(links, linkvalues))
jointposes = dict(zip(joints, values))
referenceframelist = []
i = 0
for key in lin:
    key = ReferenceFrame(key.split()[0])
    i +=1
    referenceframelist.append(key)
a = 0
referenceframelist2 = []
for key in jointposes:
    key = ReferenceFrame(key.split()[0])
    a +=1
    referenceframelist2.append(key)

base_frame = ReferenceFrame('base')
referenceframelist2.insert(0, base_frame)

print(referenceframelist)
print(referenceframelist2)

for x in jointposes:
    for i in range(6):
        jointposes[x][i] = float(jointposes[x][i])

jointangles = []
a = 1
for item in joints:
    jointangles.append(('theta' + str(a)))
    a = a+1


jointangletheta = []
for value in jointangles:
    value = dynamicsymbols(value)
    jointangletheta.append(value)


axisvalues = []
for x in joints:
    for item in root.findall('./model/'):
        if item.attrib['name'] == x:
            for child in item:
                for ax in child:
                    if child.tag == 'axis':
                        axisvalues.append(ax.text.split())
jointaxis = dict(zip(joints, axisvalues))

axval = -1
print(referenceframelist2)
for x in referenceframelist2:
    index = referenceframelist2.index(x)
    if referenceframelist2.index(x) == 0:
        pass
    else:
        if jointaxis[str(x)][0] == '1': 
            x.orient(referenceframelist2[index-1], 'Body', [jointposes[str(x)][3] + jointangletheta[axval], jointposes[str(x)][4], jointposes[str(x)][5]], 123)
        elif jointaxis[str(x)][1] == '1':
            x.orient(referenceframelist2[index-1], 'Body', [jointposes[str(x)][3], jointposes[str(x)][4] + jointangletheta[axval], jointposes[str(x)][5]], 123)
        elif jointaxis[str(x)][2] == '1':
            x.orient(referenceframelist2[index-1], 'Body', [jointposes[str(x)][3], jointposes[str(x)][4] , jointposes[str(x)][5] + jointangletheta[axval]], 123)
        else:
            x.orient(referenceframelist2[index-1], 'Body', [jointposes[str(x)][3], jointposes[str(x)][4] , jointposes[str(x)][5] + jointangletheta[axval]], 123)
        axval = axval + 1

        rotationmatrix = []

for x in referenceframelist2:
    ind = referenceframelist2.index(x)
    if referenceframelist2.index(x) == len(referenceframelist2)-1:
        pass
    else:
        rotmatrix = x.dcm(referenceframelist2[ind+1])
        rotationmatrix.append(rotmatrix.simplify())

[base, link1, link2]
[base, shoulder, joint, endeffector]
[base, shoulder, joint, endeffector]


In [2]:
rotationmatrix

[Matrix([
[                                     cos(syaw + theta3)*cos(sp),                                      -sin(syaw + theta3)*cos(sp),          sin(sp)],
[sin(syaw + theta3)*cos(sr) + sin(sp)*sin(sr)*cos(syaw + theta3), -sin(syaw + theta3)*sin(sp)*sin(sr) + cos(syaw + theta3)*cos(sr), -sin(sr)*cos(sp)],
[sin(syaw + theta3)*sin(sr) - sin(sp)*cos(syaw + theta3)*cos(sr),  sin(syaw + theta3)*sin(sp)*cos(sr) + sin(sr)*cos(syaw + theta3),  cos(sp)*cos(sr)]]), Matrix([
[                                     cos(jyaw + theta1)*cos(jp),                                      -sin(jyaw + theta1)*cos(jp),          sin(jp)],
[sin(jyaw + theta1)*cos(jr) + sin(jp)*sin(jr)*cos(jyaw + theta1), -sin(jyaw + theta1)*sin(jp)*sin(jr) + cos(jyaw + theta1)*cos(jr), -sin(jr)*cos(jp)],
[sin(jyaw + theta1)*sin(jr) - sin(jp)*cos(jyaw + theta1)*cos(jr),  sin(jyaw + theta1)*sin(jp)*cos(jr) + sin(jr)*cos(jyaw + theta1),  cos(jp)*cos(jr)]]), Matrix([
[                                     cos(eyaw + theta2)*cos(e

In [3]:
from sympy import *
init_printing(use_unicode=True)
transformationmatrix = []
i = 1
for x in rotationmatrix:
    r1 = [x[0], x[1], x[2], jointposes[str(referenceframelist2[i])][0]]
    r2 = [x[3], x[4], x[5], jointposes[str(referenceframelist2[i])][1]]
    r3 = [x[6], x[7], x[8], jointposes[str(referenceframelist2[i])][2]]
    r4 = [0, 0, 0, 1]
    
    i = i+1
    tm = Matrix([r1, r2, r3, r4])
    transformationmatrix.append(tm)
transformationmatrix

inversedjointtransform = []
for x in transformationmatrix:
    y = 1
    inversedjointtransform.append(transformationmatrix[y].inv())
    y = y+1
    
from functools import reduce
reduce(lambda x, y: x*y, inversedjointtransform)

⎡  ⎛                                                       ⎛                  
⎢  ⎜                                                       ⎝-sin(jyaw(t) + θ₁(
⎢  ⎜──────────────────────────────────────────────────────────────────────────
⎢  ⎜                                                                          
⎢  ⎜⎛   2                     2           2             2                     
⎢  ⎝⎝sin (jyaw(t) + θ₁(t))⋅sin (jp(t))⋅sin (jr(t)) + sin (jyaw(t) + θ₁(t))⋅sin
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢  ⎛                                                                          
⎢  ⎜                                                